<a href="https://colab.research.google.com/github/sandy-hermann/GEE_NDVI_Restauro/blob/main/C%C3%B3pia_de_NDVI_Restauro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NDVI com Google Earth Engine (Python)**

Por: Sandy dos Reis Hermann

26/05/2025

**Análise do monitoramento de 5 anos da cobertura vegetal em área de restauração com uso de imagens de satélite: uma abordagem usando Google Earth Engine e Google Colab.**

- Esse projeto tem como objetivo analisar por meio das geotecnologias, a cobertura vegetal da área de restauro ao entorno da PCH Anhanguera, localizada no município de São Joaquim da Barra - SP -
- Este script utiliza o Google Earth Engine para calcular o NDVI (Índice de Vegetação por Diferença Normalizada) de imagens do satélite Sentinel-2 na área de interesse com raio de 5 km, para os anos de 2018 a 2022. Os resultados são exportados automaticamente para o Google Drive.

✅ Requisitos

- Conta no Google Earth Engine (https://signup.earthengine.google.com)
- Biblioteca Python 'earthengine-api' instalada
- Autenticação feita com ee.Authenticate() e ee.Initialize()
- Permissão para gravar no seu Google Drive



Primeiro vamos importar as bibliotecas e autenticar sua conta!

In [1]:
import ee
import datetime

In [2]:
# Trigger the authentication flow
ee.Authenticate()

Nessa parte é necessário já ter entrado em sua conta do GEE e criar um novo projeto

In [3]:
 # Initialize the library
ee.Initialize(project='projeto-teste-sr-459612')

**🌍 Área de estudo - São Joaquim da Barra**

O script define um ponto central:

Latitude: -20.51016  
Longitude: -47.85615

In [4]:
# Definir o ponto de interesse e buffer de 5 km
ponto = ee.Geometry.Point([-47.85615, -20.51016])
aoi_buffer = ponto.buffer(5000)

**O que o script vai fazer?**


Exporta os resultados para uma pasta no Google Drive chamada GEE_Restauro.

1.   Remover nuvens usando a banda QA60 das imagens Sentinel-2.
2.   Calcular o NDVI: (B8 - B4) / (B8 + B4)
3. Exportar os resultados para uma pasta no Google Drive (nomear uma pasta onde você irá deixar todos os arquivos)

Os arquivos sairão em formato .tif





In [5]:
# Função para mascarar nuvens usando a banda QA60
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

In [6]:
# ⚠️ Função NDVI definida fora do loop
def calc_ndvi(img):
    return img.normalizedDifference(['B8', 'B4']).rename('NDVI')

In [7]:
# Lista de anos
anos = list(range(2018, 2023))

In [8]:
anos

[2018, 2019, 2020, 2021, 2022]

In [17]:
# Loop por ano
for ano in anos:
    inicio = ee.Date(f'{ano}-01-01')
    fim = ee.Date(f'{ano}-12-31')

    colecao = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(inicio, fim) \
        .filterBounds(aoi_buffer) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds)

    ndvi = colecao.map(calc_ndvi).mean().clip(aoi_buffer)

    task = ee.batch.Export.image.toDrive(
        image=ndvi,
        description=f'NDVI_{ano}',
        folder='GEE_Restauro',
        fileNamePrefix=f'NDVI_{ano}',
        region=aoi_buffer,
        scale=10,
        crs='EPSG:4326',
        maxPixels=1e13
    )

    task.start()
    print(f'✅ Exportação do NDVI {ano} iniciada.')

✅ Exportação do NDVI 2018 iniciada.
✅ Exportação do NDVI 2019 iniciada.
✅ Exportação do NDVI 2020 iniciada.
✅ Exportação do NDVI 2021 iniciada.
✅ Exportação do NDVI 2022 iniciada.
